# Dados Utilizados

Para a realização deste projeto, foi necessário utilizar alguns conjuntos de dados de domínio público. Nesta seção iremos entrar em mais detalhes sobre as bases utilizadas.

In [1]:
import pandas as pd
import re

### Gun Violence

DESCRIÇÃO

Fonte: [Gun Violence Data - James Ko](https://www.kaggle.com/jameslko/gun-violence-data)

In [2]:
# Load gun violence dataset
gun_violence = pd.read_csv('../databases/gun_violence.zip', compression='zip')

### State Population Totals and Components of Change: 2010-2018

DESCRIÇÃO

Fonte: [United States Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html)

In [3]:
# Load population dataset
population = pd.read_csv('../databases/nst-est2018-alldata.zip', compression='zip')

# Limpeza dos Dados

As bases de dados selecionadas possuem alguns dados nos quais não estamos interessados e que podemos remover para melhorar o desempenho e facilitar o trabalho.
Além disso algumas estão faltando informações ou possuem uma formatação não muito adequada

### Gun Violence

In [4]:
# Drop unnecessary columns
gun_violence = gun_violence[[
    'incident_id',
    'date',
    'state',
    'n_killed',
    'n_injured',
    'gun_stolen',
    'gun_type',
    'n_guns_involved',
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_type'
]]

In [5]:
# Drop missing values
gun_violence.dropna(inplace=True)
gun_violence.reset_index(drop=True, inplace=True)

In [6]:
# Change date formatting
gun_violence['year'] = pd.to_numeric(gun_violence['date'].map(lambda x: x[0:4]))
gun_violence['month'] = gun_violence['date'].map(lambda x: x[5:7])
gun_violence['day'] = gun_violence['date'].map(lambda x: x[8:10])
gun_violence.drop(columns='date', inplace=True)

In [7]:
# Drop all incidents before 2014 and after 2017
indexNames = gun_violence[(gun_violence['year'] < 2014) | (gun_violence['year'] > 2017) ].index
gun_violence.drop(indexNames, inplace=True)

In [8]:
# Change formatting on gun_stolen, gun_type, participant_age, etc.
gun_violence['gun_stolen'] = gun_violence['gun_stolen'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['gun_type'] = gun_violence['gun_type'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_age'] = gun_violence['participant_age'].map(lambda x: re.findall(r'[0-9]+', x)[1::2])
gun_violence['participant_age_group'] = gun_violence['participant_age_group'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_gender'] = gun_violence['participant_gender'].map(lambda x: re.findall(r'[a-zA-Z]+', x))
gun_violence['participant_type'] = gun_violence['participant_type'].map(lambda x: re.findall(r'[a-zA-Z-]+', x))

gun_violence.head()

,incident_id,state,n_killed,n_injured,gun_stolen,gun_type,n_guns_involved,participant_age,participant_age_group,participant_gender,participant_type,year,month,day
95,854799,Florida,0,0,[Unknown],[Unknown],1.0,"[31, 32]","[Adult, Adult]","[Male, Male]","[Subject-Suspect, Subject-Suspect]",2014,01,01
96,98134,Louisiana,1,0,[Unknown],[Unknown],1.0,"[31, 24]","[Adult, Adult]","[Male, Male]","[Victim, Subject-Suspect]",2014,01,01
97,92563,Mississippi,1,0,[Unknown],[Unknown],1.0,"[6, 13]","[Child, Teen]","[Female, Male]","[Victim, Subject-Suspect]",2014,01,01
98,97976,Hawaii,1,0,[Unknown],[Unknown],1.0,"[88, 41]","[Adult, Adult]","[Male, Male]","[Victim, Subject-Suspect]",2014,01,01
99,92117,Kentucky,0,1,[Unknown],[Unknown],1.0,[22],"[Adult, Adult]","[Male, Male]","[Victim, Subject-Suspect]",2014,01,01


### Population

In [9]:
# Drop unnecessary columns
population = population[[
    'NAME',
    'POPESTIMATE2014',
    'POPESTIMATE2015',
    'POPESTIMATE2016',
    'POPESTIMATE2017'
]]

# Rename columns
population.columns = ['state', '2014', '2015', '2016', '2017']

population = population[5:56].reset_index(drop=True)
population['mean'] = population.drop(['state'], axis=1).mean(axis=1)

population.head()

,state,2014,2015,2016,2017,mean
0,Alabama,4842481,4853160,4864745,4875120,4858876.50
1,Alaska,736307,737547,741504,739786,738786.00
2,Arizona,6733840,6833596,6945452,7048876,6890441.00
3,Arkansas,2967726,2978407,2990410,3002997,2984885.00
4,California,38625139,38953142,39209127,39399349,39046689.25


### Participantes

Vamos criar um novo data frame com informações dos participantes dos incidentes para podermos trabalhar melhor.

In [10]:
# Create new Data Frame for participants
participants = []
for index, row in gun_violence.iterrows():
    for age, group, gender, p_type in zip(row['participant_age'], row['participant_age_group'], row['participant_gender'], row['participant_type']):
        participants.append([row['incident_id'], age, group, gender, p_type])

participants = pd.DataFrame(participants, columns=['incident_id', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_type'])

participants.head()

,incident_id,participant_age,participant_age_group,participant_gender,participant_type
0,854799,31,Adult,Male,Subject-Suspect
1,854799,32,Adult,Male,Subject-Suspect
2,98134,31,Adult,Male,Victim
3,98134,24,Adult,Male,Subject-Suspect
4,92563,6,Child,Female,Victim


### Estatísticas por Estado

Vamos criar um novo data frame com estatísticas dos incidentes por Estado.

In [11]:
incidents_state = gun_violence.groupby('state')\
                  .agg({'n_injured':'sum', 'incident_id':'count', 'n_killed':'sum'})\
                  .rename(columns={'incident_id':'n_incidents'})\
                  .sort_values('state')\
                  .reset_index()

states = pd.merge(incidents_state, population[['state','mean']], on='state')
states.rename(columns={'mean': 'population'}, inplace=True)
states['injured_per_capita'] = states['n_injured']/states['population'] * 100000 / 4
states['incidents_per_capita'] = states['n_incidents']/states['population'] * 100000 / 4
states['killed_per_capita'] = states['n_killed']/states['population'] * 100000 / 4

for year in range(2014, 2018):
    incidents_year = gun_violence[gun_violence['year'] == year]
    states_year = incidents_year.groupby('state')\
                  .agg({'n_injured':'sum', 'incident_id':'count', 'n_killed':'sum'})\
                  .rename(columns={'n_injured':f'n_injured_{year}','incident_id':f'n_incidents_{year}', 'n_killed':f'n_killed_{year}'})\
                  .sort_values('state')\
                  .reset_index()
    states_year[f'injured_per_capita_{year}'] = states_year[f'n_injured_{year}']/population[f'{year}'] * 100000
    states_year[f'incidents_per_capita_{year}'] = states_year[f'n_incidents_{year}']/population[f'{year}'] * 100000
    states_year[f'killed_per_capita_{year}'] = states_year[f'n_killed_{year}']/population[f'{year}'] * 100000
    states = pd.merge(states, states_year, on='state')

states.head()

,state,n_injured,n_incidents,n_killed,population,injured_per_capita,incidents_per_capita,killed_per_capita,n_injured_2014,n_incidents_2014,...,n_killed_2016,injured_per_capita_2016,incidents_per_capita_2016,killed_per_capita_2016,n_injured_2017,n_incidents_2017,n_killed_2017,injured_per_capita_2017,incidents_per_capita_2017,killed_per_capita_2017
0,Alabama,614,1411,804,4858876.50,3.159167,7.259909,4.136759,47,124,...,234,3.885096,8.345761,4.810119,354,812,493,7.261360,16.656000,10.112572
1,Alaska,105,557,153,738786.00,3.553126,18.848489,5.177413,7,25,...,58,6.068747,28.725401,7.821940,35,198,60,4.731098,26.764497,8.110454
2,Arizona,323,776,550,6890441.00,1.171913,2.815495,1.995518,30,76,...,178,1.526179,3.426703,2.562828,156,367,252,2.213119,5.206504,3.575038
3,Arkansas,511,993,378,2984885.00,4.279897,8.316903,3.165951,26,72,...,100,4.313790,9.229504,3.344023,322,552,198,10.722621,18.381637,6.593413
4,California,1861,5180,2265,39046689.25,1.191522,3.316542,1.450187,70,219,...,641,1.548109,3.876648,1.634823,953,2636,1167,2.418822,6.690466,2.961978


### Salvar dados limpos

In [12]:
# Save clean datasets
gun_violence.to_csv('../databases/gun_violence_clean.csv', index=False)
population.to_csv('../databases/population.csv', index=False)
participants.to_csv('../databases/participants.csv', index=False)
states.to_csv('../databases/states.csv', index=False)